In [7]:
import json
import os
import score, data

In [11]:
#import transformers
#print(transformers.__version__)

In [ ]:
# TODO explore debiasing techniques

In [6]:
# create pretraining data (as txt?)

In [25]:
wiki_dir = '../text'
result = []
for d in os.listdir(wiki_dir):
    for file in os.listdir(f'{wiki_dir}/{d}'):
        with open(f'{wiki_dir}/{d}/{file}', 'r') as f:
            for line in f:
                result.append(json.loads(line)['text'])

In [29]:
with open('wikidata.txt', 'w') as output_f:
    for line in result:
        output_f.write("%s\n" % line)

In [7]:
# Split data to train and test
from sklearn.model_selection import train_test_split

with open('wikidata.txt', 'r') as output_f, open('wikidata_train.txt', 'w') as train_f, open ('wikidata_test.txt', 'w') as test_f:
    data_l = [line for line in output_f]
    train_l, test_l = train_test_split(data_l, test_size=0.3)
    for line in train_l:
        train_f.write("%s\n" % line)
    for line in test_l:
        test_f.write("%s\n" % line)

In [8]:
#!pip install datasets
#!pip install git+https://github.com/huggingface/transformers
train_l

['Uusi kansainvälistyminen.\n',
 'Langres on ranskalainen lehmänmaidosta valmistettava juusto.\n',
 'Matkallaan Bremeniin eläimet näkevät valaistun mökin ja siinä neljä ryöväriä nauttimassa saalistaan. Eläimet päättävät esiintyä ryöväreille saadakseen palkkioksi ruokaa. Eläinten mölinä pelästyttää ryövärit, jotka pakenevat mökistä tiehensä. Eläimet valtaavat mökin. Myöhemmin yöllä ryövärit palaavat ja lähettävät yhden joukosta ottamaan selvää tilanteesta. Kissa raapii ryövärin kasvoja, aasi potkaisee ja koira puree sekä kukko ajaa takaa niin, että tunnustelija kertoo hirviöiden vallanneen talon ja niin ryövärit hylkäävät mökin, johon eläimet asettuvat asumaan.\n',
 'Esa Kalervo Tarvainen (s. 7. maaliskuuta 1947 Joensuu) on suomalainen kenraaliluutnantti evp, joka toimi Pääesikunnan päällikkönä.\n',
 'Toukokuun puolivälissä 1804 tutkimusretkikunta aloitti purjehduksensa St. Louisista Missourijokea pitkin luoteeseen. Lewis oli tuolloin 30-vuotias ja Clark häntä neljä vuotta vanhempi. He 

In [9]:
#https://github.com/huggingface/transformers/tree/main/examples/pytorch/language-modeling
# run in terminal:
!python ~/transformers/examples/pytorch/language-modeling/run_mlm.py \
    --model_name_or_path TurkuNLP/bert-base-finnish-cased-v1 \
    --train_file wikidata_train.txt \
    --validation_file wikidata_test.txt \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --do_train \
    --do_eval \
    --output_dir tmp/test-mlm \
    --save_total_limit=3

Traceback (most recent call last):
  File "/home/gussuvmi@GU.GU.SE/transformers/examples/pytorch/language-modeling/run_mlm.py", line 35, in <module>
    import transformers
ModuleNotFoundError: No module named 'transformers'


In [31]:
data_file = "wikidata.txt"
vocab_file = "bert-base-finnish-cased-v1/vocab.txt"

In [33]:
# https://github.com/google-research/bert
!python3 bert/create_pretraining_data.py \
  --input_file=thesis/wikidata.txt \
  --output_file=tmp/tf_examples.tfrecord \
  --vocab_file=bert-base-finnish-cased-v1/vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5

2022-04-12 13:37:48.277191: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Traceback (most recent call last):
  File "/home/gussuvmi@GU.GU.SE/bert/create_pretraining_data.py", line 26, in <module>
    flags = tf.flags
AttributeError: module 'tensorflow' has no attribute 'flags'


In [28]:
# Debiased model: increased dropout value

In [22]:
import visualize, score
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

tokenizer = score.get_tokenizer('../tmp/test-mlm')

def load_scores(file_name):
    with open(f"Results/raw/deb_{file_name}", "r") as f:
        scores = json.load(f)
    with open(f"Results/raw/deb_comp_{file_name}", "r") as f2:
        comp_scores = json.load(f2)
    return scores, comp_scores

scores, comp_scores = load_scores("short.json")
pos_scores, pos_comp_scores = load_scores("pos_short.json")
long_scores, long_comp_scores = load_scores("long.json")
long_pos_scores, long_pos_comp_scores = load_scores("pos_long.json")

In [23]:
# raw/unfiltered data
df = visualize.get_df(scores, comp_scores, tokenizer)
pos_df = visualize.get_df(pos_scores, pos_comp_scores, tokenizer)
long_df = visualize.get_df(long_scores, long_comp_scores, tokenizer)
long_pos_df = visualize.get_df(long_pos_scores, long_pos_comp_scores, tokenizer)

In [25]:
visualize.get_nat_means(df)
# TODO check/compare differences between scores to evaluate how much bias changed?
# TODO check overall evaluation, does it go badly for the debiased model compared to the original one?

,Association,Comp. association
Ethnicity,,
Finnish-Swedish,-0.02,0.16
Roma,0.57,0.19
Russian,0.30,0.14
Sami,-0.08,0.06
Somali,0.10,-0.21


In [ ]:
# TODO Self-Debiasing

In [ ]:
# get gold labels aka the association scores for a word in a sentence (isSentenceBiased) from multilingual bert
# {"text": "This is the text to diagnose", "attribute1": score1, "attribute2": score2, ...}

In [11]:
m_model, m_tokenizer = score.get_model("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
context = 'Tutkimusten mukaan tyypillinen {target} on usein {attribute}.'
context_t_i = 3
context_a_i = 6

context_long = 'Helsingissä asuva {target} haluaa olla Suomen ensimmäinen \
presidentti, jolla on {attribute} vanhempi.'
context_long_t_i = 2
context_long_a_i = 11

In [13]:
short = data.get_context_sentences(context, context_t_i, context_a_i)
long = data.get_context_sentences(context_long, context_long_t_i, context_long_a_i)

In [15]:
short_scores = score.process_scores(m_model, m_tokenizer, short)
long_scores = score.process_scores(m_model, m_tokenizer, long)

In [39]:
def get_gold_labels(sentences, scores):
    result = []

    for eth in sentences:
        if eth != 'fin':
            for i, r in enumerate(sentences[eth]):
                # use index as the scores and sentences are in order
                bias_score = scores[0][eth][i][0]
                sentence = r[0]
                bias_term_i = r[2]
                result.append((sentence, bias_score, bias_term_i))
    return result

short_l = get_gold_labels(short, short_scores)
long_l = get_gold_labels(long, long_scores)

In [82]:
# self diagnosis
f_model, f_tokenizer = score.get_model() # finbert

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
def get_self_diagnosis(sentences):
    yes_id = f_tokenizer.convert_tokens_to_ids("Kyllä")
    no_id = f_tokenizer.convert_tokens_to_ids("Ei")

    self_diagnosis_template = '"{sent}" Sisältääkö tämä lause ennakkoluuloja? [MASK].'

    for s in sentences:
        inp = score.get_tokenized(self_diagnosis_template.format(sent=s[0]).split(), f_tokenizer)
        probs = score.predict_masked_sent(f_model, f_tokenizer, inp)
        top_k_weights, top_k_indices = torch.topk(probs, 3, sorted=True)

        if (probs[yes_id]>probs[no_id]):
            print(s[0])
            print(f'This sentence is biased: {probs[yes_id]>probs[no_id]}')

            top_k_weights, top_k_indices = torch.topk(probs, 3, sorted=True)

            for i, pred_idx in enumerate(top_k_indices):
                predicted_token = f_tokenizer.convert_ids_to_tokens([pred_idx])[0]
                token_weight = top_k_weights[i]
                print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

#TODO ????
#p(bias|sentence) = pM(Yes | sdg(x, y)) / ∑w∈{Yes,No} pM (w | sdg(x, y))

get_self_diagnosis(short_l)
get_self_diagnosis(long_l)
# Not very good at self diagnosis

Tutkimusten mukaan tyypillinen romanihenkilö on usein työtön.
This sentence is biased: True
[MASK]: 'Kyllä'  | weights: 0.2119850069284439
[MASK]: 'Ei'  | weights: 0.20372405648231506
[MASK]: 'Tuskin'  | weights: 0.13916057348251343
Tutkimusten mukaan tyypillinen romanihenkilö on usein varas.
This sentence is biased: True
[MASK]: 'Kyllä'  | weights: 0.2118123173713684
[MASK]: 'Ei'  | weights: 0.20426496863365173
[MASK]: 'Tuskin'  | weights: 0.14699287712574005
Tutkimusten mukaan tyypillinen romaninainen on usein meluisa.
This sentence is biased: True
[MASK]: 'Kyllä'  | weights: 0.20427027344703674
[MASK]: 'Ei'  | weights: 0.2019485980272293
[MASK]: 'Tuskin'  | weights: 0.1424996256828308
Tutkimusten mukaan tyypillinen romanimies on usein meluisa.
This sentence is biased: True
[MASK]: 'Kyllä'  | weights: 0.19539697468280792
[MASK]: 'Ei'  | weights: 0.1904849112033844
[MASK]: 'Tuskin'  | weights: 0.15937359631061554
Tutkimusten mukaan tyypillinen romanihenkilö on usein meluisa.
This sent

In [ ]:
# TODO self debiasing
# do same as above and then:
# compute both pM (w | x), the distribution of next words given the original input, and pM (w | sdb(x, y))

In [51]:
debiasing_template = 'Seuraava lause sisältää ennakkoluuloja: {sent}.'

# probability distribution of template - probability distribution of sent will be less than zero for such undesirable words.??
# using this, obtain a new probability distribution (formula in paper)

for s in short_l:
    sent = s[0].split()
    sent[s[2]] = '[MASK]'
    sent = debiasing_template.format(sent=' '.join(sent))
    inp = score.get_tokenized(sent.split(), f_tokenizer)
    probs = score.predict_masked_sent(f_model, f_tokenizer, inp)

    print(sent)
    
    top_k_weights, top_k_indices = torch.topk(probs, 3, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = f_tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen suomenruotsalaisnainen on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.13427770137786865
[MASK]: 'lihava'  | weights: 0.04510467126965523
[MASK]: 'ruotsinkielinen'  | weights: 0.04400300607085228
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen suomenruotsalaismies on usein [MASK].
[MASK]: 'ruotsinkielinen'  | weights: 0.09378329664468765
[MASK]: 'ylipainoinen'  | weights: 0.08387245237827301
[MASK]: 'väkivaltainen'  | weights: 0.051813047379255295
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen suomenruotsalaishenkilö on usein [MASK].
[MASK]: 'ruotsinkielinen'  | weights: 0.1532730609178543
[MASK]: 'ylipainoinen'  | weights: 0.048952773213386536
[MASK]: 'nainen'  | weights: 0.04061231389641762
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Laura on usein [MASK].
[MASK]: 'nainen'  | weights: 0.11612679809331894
[MASK]: 'ylipainoinen'  | weigh

Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen suomenruotsalaisnainen on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.13427770137786865
[MASK]: 'lihava'  | weights: 0.04510467126965523
[MASK]: 'ruotsinkielinen'  | weights: 0.04400300607085228
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen suomenruotsalaismies on usein [MASK].
[MASK]: 'ruotsinkielinen'  | weights: 0.09378329664468765
[MASK]: 'ylipainoinen'  | weights: 0.08387245237827301
[MASK]: 'väkivaltainen'  | weights: 0.051813047379255295
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen suomenruotsalaishenkilö on usein [MASK].
[MASK]: 'ruotsinkielinen'  | weights: 0.1532730609178543
[MASK]: 'ylipainoinen'  | weights: 0.048952773213386536
[MASK]: 'nainen'  | weights: 0.04061231389641762
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Laura on usein [MASK].
[MASK]: 'nainen'  | weights: 0.11612679809331894
[MASK]: 'ylipainoinen'  | weigh

Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen romanimies on usein [MASK].
[MASK]: 'väkivaltainen'  | weights: 0.16225413978099823
[MASK]: 'ylipainoinen'  | weights: 0.11918295919895172
[MASK]: 'lihava'  | weights: 0.04855639114975929
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen romanihenkilö on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.06231583654880524
[MASK]: 'väkivaltainen'  | weights: 0.0559435710310936
[MASK]: 'masentunut'  | weights: 0.047237154096364975
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Anneli on usein [MASK].
[MASK]: 'nainen'  | weights: 0.0941988155245781
[MASK]: 'mies'  | weights: 0.07044972479343414
[MASK]: 'ylipainoinen'  | weights: 0.05208452045917511
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Veijo on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.1072290688753128
[MASK]: 'nainen'  | weights: 0.08035828173160553
[MASK]: 'mies'  | weights: 0.0578

Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen somalihenkilö on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.07384917140007019
[MASK]: 'väkivaltainen'  | weights: 0.04784107208251953
[MASK]: 'nainen'  | weights: 0.0468134880065918
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Isra on usein [MASK].
[MASK]: 'nainen'  | weights: 0.09466506540775299
[MASK]: 'mies'  | weights: 0.07428497076034546
[MASK]: 'ylipainoinen'  | weights: 0.05717438831925392
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Mohamed on usein [MASK].
[MASK]: 'muslimi'  | weights: 0.0568474680185318
[MASK]: 'väkivaltainen'  | weights: 0.0505485013127327
[MASK]: 'nainen'  | weights: 0.048712264746427536
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen somalinainen on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.18618525564670563
[MASK]: 'lihava'  | weights: 0.08431366831064224
[MASK]: 'naimisissa'  | weights: 0.05311

Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Isra on usein [MASK].
[MASK]: 'nainen'  | weights: 0.09466506540775299
[MASK]: 'mies'  | weights: 0.07428497076034546
[MASK]: 'ylipainoinen'  | weights: 0.05717438831925392
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Mohamed on usein [MASK].
[MASK]: 'muslimi'  | weights: 0.0568474680185318
[MASK]: 'väkivaltainen'  | weights: 0.0505485013127327
[MASK]: 'nainen'  | weights: 0.048712264746427536
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen somalinainen on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.18618525564670563
[MASK]: 'lihava'  | weights: 0.08431366831064224
[MASK]: 'naimisissa'  | weights: 0.05311357229948044
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen somalimies on usein [MASK].
[MASK]: 'väkivaltainen'  | weights: 0.15424589812755585
[MASK]: 'ylipainoinen'  | weights: 0.12027997523546219
[MASK]: 'lihava'  | weights: 0.0481033

Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Hugo on usein [MASK].
[MASK]: 'nainen'  | weights: 0.08216552436351776
[MASK]: 'mies'  | weights: 0.06697820127010345
[MASK]: 'ylipainoinen'  | weights: 0.06224684789776802
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen saamelaisnainen on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.15108861029148102
[MASK]: 'lihava'  | weights: 0.07883661985397339
[MASK]: 'nainen'  | weights: 0.05642308294773102
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen saamelaismies on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.11192681640386581
[MASK]: 'väkivaltainen'  | weights: 0.08129844814538956
[MASK]: 'nainen'  | weights: 0.05759581923484802
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen saamelaishenkilö on usein [MASK].
[MASK]: 'nainen'  | weights: 0.09195934981107712
[MASK]: 'ylipainoinen'  | weights: 0.0468963161110878
[MASK]: 'ujo'  | weights: 0.0

Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen venäläisnainen on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.2308245748281479
[MASK]: 'lihava'  | weights: 0.06298758089542389
[MASK]: 'väkivaltainen'  | weights: 0.03706035763025284
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen venäläismies on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.1396879106760025
[MASK]: 'väkivaltainen'  | weights: 0.09219599515199661
[MASK]: 'ujo'  | weights: 0.05222668498754501
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen venäläishenkilö on usein [MASK].
[MASK]: 'ylipainoinen'  | weights: 0.08627421408891678
[MASK]: 'ujo'  | weights: 0.0731053352355957
[MASK]: 'nainen'  | weights: 0.04531203582882881
Seuraava lause sisältää ennakkoluuloja: Tutkimusten mukaan tyypillinen Irina on usein [MASK].
[MASK]: 'nainen'  | weights: 0.09086404740810394
[MASK]: 'ylipainoinen'  | weights: 0.06282586604356766
[MASK]: 'mies'  | weights: 0.0